In [2]:
import httplib2
from bs4 import BeautifulSoup, SoupStrainer
import requests

import time

from PyPDF2 import PdfFileReader
import io

import numpy as np
import pandas as pd
import re
import array
from time import sleep

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

In [3]:
# click privacy policy button, pass in webdriver and data safety urls with privacy policy button on page
def press_button(driver, data_url):
    driver.implicitly_wait(1)
    driver.get(data_url)
    print(data_url)
    # press privacy policy button by LINK_TEXT from html/web inspection page
    
    try:
        button = driver.find_element(By.LINK_TEXT, "privacy policy")
        button.click()
    
    except NoSuchElementException:
        pass
        current = "N/A"
        print(current)
        return current
    
    
    # close unused tabs, url for privacy policy is child window
#     p = driver.window_handles[0]
#     c = driver.window_handles[1]
#     driver.switch_to.window(c)
    
#     WebDriverWait(driver, 2)
    
#     current = driver.current_url
#     print(current)
    comp = driver.current_url
    tabs_open = len(driver.window_handles)

    if tabs_open > 1:        
        p = driver.window_handles[0]
        c = driver.window_handles[1]
        driver.switch_to.window(c)

        WebDriverWait(driver, 4)

        current = driver.current_url
        print(current)
        # close child window and switch to original/parent window
        driver.close()
    
        driver.switch_to.window(p)
    else:
        WebDriverWait(driver, 2)
        current = driver.current_url
        print(current)

    sleep(1)

    # return privacy policy url for df
    return current

In [4]:
# scroll search page to get all returned app url's and id's
def scroll(driver, timeout):
    last_height = driver.execute_script("return document.body.scrollHeight")
    scroll_pause_time = timeout
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(scroll_pause_time)
        
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
            
        last_height = new_height

In [5]:
# get urls from search
def get_urls(starts_with):
    http = httplib2.Http()
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--incognito")
    driver = webdriver.Chrome()
    # wait for page to load
    driver.implicitly_wait(30)
    # provide url(s) for driver
    driver.get("https://play.google.com/store/search?q=" + starts_with  + "&c=apps")
    # scroll to get all apps
    scroll(driver, 2)

    # read and obtain all urls
    soup_a = BeautifulSoup(driver.page_source, 'lxml')
    
    driver.close()

    # copy all urls
    urls = []
    for link in soup_a.find_all('a'):
        urls.append(link.get('href'))
    return urls

In [6]:
# alphabet = "abcdefghijklmnopqrstuvwxyz"
# categories = ["Daydream", "Beauty", "Auto & Vehicles", "Business", "Comics", "Books & Reference", "Dating", "Education", "Communication", "Food & Drink", "Entertainment", "Health & Fitness", "Events", "Finance", "Libraries & Demo", "House & Home", "Lifestyle", "Medical", "Maps & Navigation", "News & Magazines", "Audio", "Personalization", "Photography", "Parenting", "Shopping", "Productivity", "Tools", "Social", "Sports", "Video Players & Editors", "Travel & Local", "Weather",
#               "Adventure", "Arcade", "Action", "Board", "Casino", "Casual", "Card", "Music", "Puzzle", "Educational", "Role Playing", "Racing", "Sports", "Strategy", "Simulation", "Brain Games", "Trivia", "Word", "Education", "Creativity", "Video", "TV", "Weather", "a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", "p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z"]
categories = ['word']
urls = []
# alphabet = 'ab'

# loop through search categories and add to urls array
for char_index in range(len(categories)):
    print(categories[char_index])
    urls += get_urls(categories[char_index])
    print(urls[char_index])
print(len(urls))
for i in range(len(urls)):
    print(urls[i])


word
/store/games
46
/store/games
https://policies.google.com/privacy
https://myaccount.google.com/termsofservice
/store/search?q=word&c=apps
/store/search?q=word&c=movies
/store/search?q=word&c=books
/store/apps/details?id=com.microsoft.office.word
/store/apps/details?id=com.google.android.apps.docs.editors.docs
/store/apps/details?id=com.palmmob.office2global
/store/apps/details?id=com.officedocument.word.docx.document.viewer
/store/apps/details?id=cn.wps.moffice_eng
/store/apps/details?id=com.peoplefun.wordcross
/store/apps/details?id=com.officesuite.fileopener.fileviewer.all.document.reader.word.excel.pptx.pdf
/store/apps/details?id=com.microsoft.office.officehubrow
/store/apps/details?id=com.bitmango.go.wordcookies
/store/apps/details?id=ccom.legend.office.viewer
/store/apps/details?id=com.mobisystems.office
/store/apps/details?id=com.wordgame.words.connect
/store/apps/details?id=com.oneup.word.search
/store/apps/details?id=com.platinumplayer.word.addict
/store/apps/details?id=com

In [7]:
def get_data_safety_urls(app_ids):
    
    base_url = "https://play.google.com/store/apps/datasafety?id="
    data_urls = []
    
    data_urls = [base_url + s for s in app_ids]
    
    return data_urls
            

In [8]:
# remove extra characters from urls array to get only app ids
app_ids = [val for val in urls if val.startswith('/store/apps/details?id=')]

for i in range(len(app_ids)):
    app_ids[i] = app_ids[i][23:]
    print(app_ids[i])
print(len(app_ids))
print("------------------------------------------------------------------------------------------------------------")
data_urls = get_data_safety_urls(app_ids)
print(len(data_urls))
print(data_urls, app_ids)

com.microsoft.office.word
com.google.android.apps.docs.editors.docs
com.palmmob.office2global
com.officedocument.word.docx.document.viewer
cn.wps.moffice_eng
com.peoplefun.wordcross
com.officesuite.fileopener.fileviewer.all.document.reader.word.excel.pptx.pdf
com.microsoft.office.officehubrow
com.bitmango.go.wordcookies
ccom.legend.office.viewer
com.mobisystems.office
com.wordgame.words.connect
com.oneup.word.search
com.platinumplayer.word.addict
com.zynga.words3
com.zynga.words
com.fugo.wow
in.playsimple.wordtrip
com.zynga.wwf2.free
com.tangramgames.gourddoll.wordcrush
com.pieyel.scrabble
com.marulgames.wordsurf
word.connect.wordswipe.wordstacks.wordstory.wordscapes.word.games
23
------------------------------------------------------------------------------------------------------------
23
['https://play.google.com/store/apps/datasafety?id=com.microsoft.office.word', 'https://play.google.com/store/apps/datasafety?id=com.google.android.apps.docs.editors.docs', 'https://play.google.com/

In [9]:
# pd.set_option('max_colwidth', 400)
# df = pd.DataFrame(data_urls, columns=['data safety urls'])
# display(df)

In [10]:
# appid_df = pd.DataFrame(app_ids, columns=['app id'])
# display(appid_df)

In [11]:
# concatenate app id and data safety urls dfs
df = pd.DataFrame(data_urls, columns=['data safety urls'])
appid_df = pd.DataFrame(app_ids, columns=['app id'])
pd.set_option('max_colwidth', 400)
all_df = pd.concat([appid_df, df], axis=1, ignore_index=True)

all_df.columns = ['app id', 'data safety url']
display(all_df)

,app id,data safety url
0,com.microsoft.office.word,https://play.google.com/store/apps/datasafety?id=com.microsoft.office.word
1,com.google.android.apps.docs.editors.docs,https://play.google.com/store/apps/datasafety?id=com.google.android.apps.docs.editors.docs
2,com.palmmob.office2global,https://play.google.com/store/apps/datasafety?id=com.palmmob.office2global
3,com.officedocument.word.docx.document.viewer,https://play.google.com/store/apps/datasafety?id=com.officedocument.word.docx.document.viewer
4,cn.wps.moffice_eng,https://play.google.com/store/apps/datasafety?id=cn.wps.moffice_eng
5,com.peoplefun.wordcross,https://play.google.com/store/apps/datasafety?id=com.peoplefun.wordcross
6,com.officesuite.fileopener.fileviewer.all.document.reader.word.excel.pptx.pdf,https://play.google.com/store/apps/datasafety?id=com.officesuite.fileopener.fileviewer.all.document.reader.word.excel.pptx.pdf
7,com.microsoft.office.officehubrow,https://play.google.com/store/apps/datasafety?id=com.microsoft.office.officehubrow
8,com.bitmango.go.wordcookies,https://play.google.com/store/apps/datasafety?id=com.bitmango.go.wordcookies
9,ccom.legend.office.viewer,https://play.google.com/store/apps/datasafety?id=ccom.legend.office.viewer


In [12]:
darr = all_df['data safety url'].to_numpy()
print(darr)

['https://play.google.com/store/apps/datasafety?id=com.microsoft.office.word'
 'https://play.google.com/store/apps/datasafety?id=com.google.android.apps.docs.editors.docs'
 'https://play.google.com/store/apps/datasafety?id=com.palmmob.office2global'
 'https://play.google.com/store/apps/datasafety?id=com.officedocument.word.docx.document.viewer'
 'https://play.google.com/store/apps/datasafety?id=cn.wps.moffice_eng'
 'https://play.google.com/store/apps/datasafety?id=com.peoplefun.wordcross'
 'https://play.google.com/store/apps/datasafety?id=com.officesuite.fileopener.fileviewer.all.document.reader.word.excel.pptx.pdf'
 'https://play.google.com/store/apps/datasafety?id=com.microsoft.office.officehubrow'
 'https://play.google.com/store/apps/datasafety?id=com.bitmango.go.wordcookies'
 'https://play.google.com/store/apps/datasafety?id=ccom.legend.office.viewer'
 'https://play.google.com/store/apps/datasafety?id=com.mobisystems.office'
 'https://play.google.com/store/apps/datasafety?id=com.wo

In [13]:
# press privacy policy button on page
driver = webdriver.Chrome()
driver.implicitly_wait(2)
temp = []
df = pd.DataFrame(columns=['privacy_policy_url'])
for i in darr:
#     temp += press_button(driver, i)
    df = df.append({'privacy_policy_url':press_button(driver,i)}, ignore_index=True)
#     urls = urls.append({'privacy_policy_url': str(priv_url)}, ignore_index=True)
#     print(i)
# urls = url.concat(urls, df)
# df.insert(loc=0, column = 'privacy_policy_url', value=temp)
driver.close()
# display(urls)

https://play.google.com/store/apps/datasafety?id=com.microsoft.office.word
2
http://www.google.com/url?sa=j&url=http%3A%2F%2Fgo.microsoft.com%2Ffwlink%2F%3FLinkID%3D507380&uct=1665955836&usg=r6hIw-AYgrrIKKfFTzDf-d3Nrm0.


/var/folders/st/293fz_dj6651kb2w_fsgr6fx7894dv/T/ipykernel_69222/3170996930.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'privacy_policy_url':press_button(driver,i)}, ignore_index=True)


https://play.google.com/store/apps/datasafety?id=com.google.android.apps.docs.editors.docs
1
https://policies.google.com/privacy


/var/folders/st/293fz_dj6651kb2w_fsgr6fx7894dv/T/ipykernel_69222/3170996930.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'privacy_policy_url':press_button(driver,i)}, ignore_index=True)


https://play.google.com/store/apps/datasafety?id=com.palmmob.office2global
2
http://www.google.com/url?sa=j&url=http%3A%2F%2Fwww.palmmob.cn%2Fyinsi%2Fword2globalyinsi.html&uct=1665955836&usg=dbHKwAw00vLBBfNAp9Emmd8vapw.


...

/var/folders/st/293fz_dj6651kb2w_fsgr6fx7894dv/T/ipykernel_69222/3170996930.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'privacy_policy_url':press_button(driver,i)}, ignore_index=True)


https://play.google.com/store/apps/datasafety?id=com.marulgames.wordsurf
2
http://marul.games/privacy_policy/word_surf_pp.html


/var/folders/st/293fz_dj6651kb2w_fsgr6fx7894dv/T/ipykernel_69222/3170996930.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'privacy_policy_url':press_button(driver,i)}, ignore_index=True)


https://play.google.com/store/apps/datasafety?id=word.connect.wordswipe.wordstacks.wordstory.wordscapes.word.games
2
http://52.52.138.46/support/fantasyword/wordcross/policy.html


/var/folders/st/293fz_dj6651kb2w_fsgr6fx7894dv/T/ipykernel_69222/3170996930.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'privacy_policy_url':press_button(driver,i)}, ignore_index=True)


In [14]:
display(df)

,privacy_policy_url
0,http://www.google.com/url?sa=j&url=http%3A%2F%2Fgo.microsoft.com%2Ffwlink%2F%3FLinkID%3D507380&uct=1665955836&usg=r6hIw-AYgrrIKKfFTzDf-d3Nrm0.
1,https://policies.google.com/privacy
2,http://www.google.com/url?sa=j&url=http%3A%2F%2Fwww.palmmob.cn%2Fyinsi%2Fword2globalyinsi.html&uct=1665955836&usg=dbHKwAw00vLBBfNAp9Emmd8vapw.
3,N/A
4,N/A
5,http://www.google.com/url?sa=j&url=http%3A%2F%2Fwww.peoplefun.com%2Fprivacy&uct=1665955836&usg=rcnsoveKp5jD8qsk1oEKfPeRU58.
6,N/A
7,http://www.google.com/url?sa=j&url=http%3A%2F%2Fgo.microsoft.com%2Ffwlink%2F%3FLinkID%3D507380&uct=1665955836&usg=r6hIw-AYgrrIKKfFTzDf-d3Nrm0.
8,https://www.bitmango.com/privacy-policy/
9,N/A


In [15]:
display(all_df)

,app id,data safety url
0,com.microsoft.office.word,https://play.google.com/store/apps/datasafety?id=com.microsoft.office.word
1,com.google.android.apps.docs.editors.docs,https://play.google.com/store/apps/datasafety?id=com.google.android.apps.docs.editors.docs
2,com.palmmob.office2global,https://play.google.com/store/apps/datasafety?id=com.palmmob.office2global
3,com.officedocument.word.docx.document.viewer,https://play.google.com/store/apps/datasafety?id=com.officedocument.word.docx.document.viewer
4,cn.wps.moffice_eng,https://play.google.com/store/apps/datasafety?id=cn.wps.moffice_eng
5,com.peoplefun.wordcross,https://play.google.com/store/apps/datasafety?id=com.peoplefun.wordcross
6,com.officesuite.fileopener.fileviewer.all.document.reader.word.excel.pptx.pdf,https://play.google.com/store/apps/datasafety?id=com.officesuite.fileopener.fileviewer.all.document.reader.word.excel.pptx.pdf
7,com.microsoft.office.officehubrow,https://play.google.com/store/apps/datasafety?id=com.microsoft.office.officehubrow
8,com.bitmango.go.wordcookies,https://play.google.com/store/apps/datasafety?id=com.bitmango.go.wordcookies
9,ccom.legend.office.viewer,https://play.google.com/store/apps/datasafety?id=ccom.legend.office.viewer


In [30]:
print(len(df.index))
print(len(all_df.index))


merged = pd.concat([df, all_df], axis=1)
display(merged)

23
23


,privacy_policy_url,one,app id,data safety url,one
0,http://www.google.com/url?sa=j&url=http%3A%2F%2Fgo.microsoft.com%2Ffwlink%2F%3FLinkID%3D507380&uct=1665955836&usg=r6hIw-AYgrrIKKfFTzDf-d3Nrm0.,1,com.microsoft.office.word,https://play.google.com/store/apps/datasafety?id=com.microsoft.office.word,1
1,https://policies.google.com/privacy,1,com.google.android.apps.docs.editors.docs,https://play.google.com/store/apps/datasafety?id=com.google.android.apps.docs.editors.docs,1
2,http://www.google.com/url?sa=j&url=http%3A%2F%2Fwww.palmmob.cn%2Fyinsi%2Fword2globalyinsi.html&uct=1665955836&usg=dbHKwAw00vLBBfNAp9Emmd8vapw.,1,com.palmmob.office2global,https://play.google.com/store/apps/datasafety?id=com.palmmob.office2global,1
3,N/A,1,com.officedocument.word.docx.document.viewer,https://play.google.com/store/apps/datasafety?id=com.officedocument.word.docx.document.viewer,1
4,N/A,1,cn.wps.moffice_eng,https://play.google.com/store/apps/datasafety?id=cn.wps.moffice_eng,1
5,http://www.google.com/url?sa=j&url=http%3A%2F%2Fwww.peoplefun.com%2Fprivacy&uct=1665955836&usg=rcnsoveKp5jD8qsk1oEKfPeRU58.,1,com.peoplefun.wordcross,https://play.google.com/store/apps/datasafety?id=com.peoplefun.wordcross,1
6,N/A,1,com.officesuite.fileopener.fileviewer.all.document.reader.word.excel.pptx.pdf,https://play.google.com/store/apps/datasafety?id=com.officesuite.fileopener.fileviewer.all.document.reader.word.excel.pptx.pdf,1
7,http://www.google.com/url?sa=j&url=http%3A%2F%2Fgo.microsoft.com%2Ffwlink%2F%3FLinkID%3D507380&uct=1665955836&usg=r6hIw-AYgrrIKKfFTzDf-d3Nrm0.,1,com.microsoft.office.officehubrow,https://play.google.com/store/apps/datasafety?id=com.microsoft.office.officehubrow,1
8,https://www.bitmango.com/privacy-policy/,1,com.bitmango.go.wordcookies,https://play.google.com/store/apps/datasafety?id=com.bitmango.go.wordcookies,1
9,N/A,1,ccom.legend.office.viewer,https://play.google.com/store/apps/datasafety?id=ccom.legend.office.viewer,1


In [18]:
# driver = webdriver.Chrome()
# driver.implicitly_wait(2)
# temp = press_button(driver, "https://play.google.com/store/apps/datasafety?id=com.apple.atve.androidtv.appletv")
# df.insert(loc=1, column = 'privacy policy url', value=temp)
# driver.close()

https://play.google.com/store/apps/datasafety?id=com.apple.atve.androidtv.appletv
https://www.apple.com/legal/privacy/en-ww/


In [64]:
link = "https://www.apple.com/legal/privacy/pdfs/apple-privacy-policy-en-ww.pdf"
policy = []

r = requests.get(link)
f = io.BytesIO(r.content)

reader = PdfFileReader(f)
count = reader.numPages

for i in range(count):
    policy += reader.getPage(i).extractText().split('\n')

In [65]:
print(policy)

['Apple Privacy Policy Apple’s Privacy Policy describes how Apple collects, uses, and shares your personal data. Updated October 27, 2021  ', 'In addition to this Privacy Policy, we provide data and privacy information embedded in our products and certain features that ask to use your personal information. This product-specific information is accompanied by our Data & Privacy Icon.  You will be given an opportunity to review this product-specific information before using these features. You also can view this information at any time, either in settings related to those features and/or online at apple.com/legal/privacy/data.  Please take a moment to familiarize yourself with our privacy practices, accessible via the headings below, and contact us if you have questions. Your California Privacy Disclosures Information Regarding Commercial Electronic Messages in CanadaApple Health Study Apps Privacy Policy ', '', 'What Is Personal Data at Apple? At Apple, we believe strongly in fundamental

In [66]:
# begin working sample of reading privacy policy html page with no dropdown menu's

# url = "https://snap.com/en-US/privacy/privacy-policy"
url = "https://www.whatsapp.com/legal/privacy-policy"

res = requests.get(url)
html_page = res.content

soup = BeautifulSoup(html_page, 'html.parser')
text = soup.find_all(text=True)

In [67]:
set([t.parent.name for t in text])

{'[document]',
 'a',
 'b',
 'body',
 'h1',
 'h3',
 'h4',
 'h5',
 'h6',
 'head',
 'html',
 'i',
 'li',
 'option',
 'p',
 'script',
 'span',
 'title'}

In [100]:
output = ''
blacklist = [
    '[document]',
   'noscript',
    'header',
    'html',
    'meta',
    'head', 
    'input',
    'script',
    'link',
    'style'
]

for t in text:
    if t.parent.name not in blacklist:
        output += '{} '.format(t)

In [101]:
print(output)

# end html page read

Privacy Policy Privacy Policy Select your language azərbaycan Afrikaans Bahasa Indonesia Melayu català čeština dansk Deutsch eesti English español français Gaeilge hrvatski italiano Kiswahili latviešu lietuvių magyar Nederlands norsk bokmål o‘zbek Filipino polski Português (Brasil) Português (Portugal) română shqip slovenčina slovenščina suomi svenska Tiếng Việt Türkçe Ελληνικά български қазақ тілі македонски русский српски українська עברית العربية فارسی اردو বাংলা हिन्दी ગુજરાતી ಕನ್ನಡ मराठी ਪੰਜਾਬੀ தமிழ் తెలుగు മലയാളം ไทย 简体中文 繁體中文（台灣） 繁體中文（香港） 日本語 한국어 Download Features Privacy Help Center Get in touch azərbaycan Afrikaans Bahasa Indonesia Melayu català čeština dansk Deutsch eesti English español français Gaeilge hrvatski italiano Kiswahili latviešu lietuvių magyar Nederlands norsk bokmål o‘zbek Filipino polski Português (Brasil) Português (Portugal) română shqip slovenčina slovenščina suomi svenska Tiếng Việt Türkçe Ελληνικά български қазақ тілі македонски русский српски українська עב

In [31]:
pwd

'/Users/allison.lu'